# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 100000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Инструкция по выполнению проекта**

Загрузите и подготовьте данные. Поясните порядок действий.

Обучите и проверьте модель для каждого региона:
 - 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
 - 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
 - 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
 - 2.4. Напечатайте на экране средний запас сырья и RMSE модели.
 - 2.5. Проанализируйте результаты.
Подготовьтесь к расчёту прибыли:
 - 3.1. Сохраните в коде все ключевые значения для расчётов.
 - 3.2. Посчитайте минимальный средний объём сырья в месторождениях региона, достаточный для его разработки. Напишите выводы.
 - 3.3. Напишите функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.
Посчитайте риски и прибыль для каждого региона:
 - 4.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 - 4.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков.
 - 4.3. Напишите выводы: предложите регион для разработки месторождений и обоснуйте выбор.

**Описание данных**
Данные геологоразведки трёх регионов находятся в файлах:
 - geo_data_0.csv.
 - geo_data_1.csv.
 - geo_data_2.csv.
 - id — уникальный идентификатор месторождения;
 - f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
 - product — объём запасов в месторождении (тыс. баррелей).

**Условия задачи:**
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона проводится исследование 500 точек.
Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.
Один баррель сырья приносит 4500 рублей прибыли.
Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

Откроем и изучим общую информацию о данных.

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
display(data_0.head())
display(data_0.info())
print(data_0['id'].duplicated().sum())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

10


### Вывод
Файлы открыты, пропущенных значений нет, но есть строки с повторяющимися id, причины их возникновения могут быть разными(ошибка в записи id или данные были получены из разных источников), но т.к. таких строк немного(в первом файле всего 10), то было решено от них избавиться.

Создадим функцию для обработки входных данных. preproc - функция, которая избавляется от строк с дубликатами id, затем удаляет стоблец с id(т.к. он не будет использоваться для построения модели), а также производит разбиение на обучающую и валидационную выборки.

In [3]:
def preproc(data):
    data['id'] = data['id'].drop_duplicates().reset_index(drop = True)
    data.dropna(inplace = True)
    
    data.drop(data[['id']],axis = 1, inplace = True)
    
    data_train_cur, data_valid_cur = train_test_split(data, test_size = 0.25, random_state = 12345)

    features_train_cur = data_train_cur.drop(['product'], axis = 1)
    target_train_cur = data_train_cur['product']

    features_valid_cur = data_valid_cur.drop(['product'], axis = 1)
    target_valid_cur = data_valid_cur['product']
    
    return features_train_cur, target_train_cur, features_valid_cur, target_valid_cur

In [4]:
features_train_0, target_train_0, features_valid_0, target_valid_0 = preproc(data_0)
features_train_1, target_train_1, features_valid_1, target_valid_1 = preproc(data_1)
features_train_2, target_train_2, features_valid_2, target_valid_2 = preproc(data_2)

In [5]:
print(features_train_0.shape, target_train_0.shape, features_valid_0.shape, target_valid_0.shape)
print(features_train_1.shape, target_train_1.shape, features_valid_1.shape, target_valid_1.shape)
print(features_train_2.shape, target_train_2.shape, features_valid_2.shape, target_valid_2.shape)

(74992, 3) (74992,) (24998, 3) (24998,)
(74997, 3) (74997,) (24999, 3) (24999,)
(74997, 3) (74997,) (24999, 3) (24999,)


После того, как получены выборки перейдем к масштабированию количественных признаков. Функция scale проводит стандартизацию признаков.

In [6]:
numeric = ['f0','f1','f2']
def scale(features_train,features_valid):
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train_scaled = scaler.transform(features_train[numeric])
    features_valid_scaled = scaler.transform(features_valid[numeric])
    return features_train_scaled, features_valid_scaled

In [7]:
features_train_0, features_valid_0 = scale(features_train_0, features_valid_0)
features_train_1, features_valid_1 = scale(features_train_1, features_valid_1)
features_train_2, features_valid_2 = scale(features_train_2, features_valid_2)

### Вывод
Файл изучен, данные разбиты на выборки и отмасштабированы.

## Обучение и проверка модели

Для обучения моделей будет использоваться функция model_fitting, которая обучает модель линейной регрессии и возвращает метрики для обучающей и тестовой выборки: R2 и rmse.

In [8]:
def model_fitting(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    mse_train = mean_squared_error(target_train, model.predict(features_train))
    rmse_train = mse_train ** (0.5)
    mse_valid = mean_squared_error(target_valid, predictions)
    rmse_valid = mse_valid ** (0.5)
    print('Обучающая выборка: Среднее значение обучающей выборки: {}'.format(target_train.mean()))
    print('Обучающая выборка: rmse: {} R2: {}'.format(rmse_train, model.score(features_train, target_train)))
    print('Валидационная выборка: rmse: {} R2: {}'.format(rmse_valid, model.score(features_valid, target_valid)))
    predictions = pd.Series(predictions)
    return predictions, rmse_valid

Перейдем к обучению моделей для различных регионов. 

### Первый регион

In [9]:
predictions_0, rmse_0 = model_fitting(features_train_0, target_train_0, features_valid_0, target_valid_0)

Обучающая выборка: Среднее значение обучающей выборки: 92.55950285453767
Обучающая выборка: rmse: 37.730311817055494 R2: 0.2755826069630001
Валидационная выборка: rmse: 37.580631363701684 R2: 0.2760169220348231


### Второй регион

In [10]:
predictions_1, rmse_1 = model_fitting(features_train_1, target_train_1, features_valid_1, target_valid_1)

Обучающая выборка: Среднее значение обучающей выборки: 68.88032559052276
Обучающая выборка: rmse: 0.8906292533412161 R2: 0.9996242014248895
Валидационная выборка: rmse: 0.8896192801777811 R2: 0.9996250751075333


### Третий регион

In [11]:
predictions_2, rmse_2 = model_fitting(features_train_2, target_train_2, features_valid_2, target_valid_2)

Обучающая выборка: Среднее значение обучающей выборки: 94.9993136587081
Обучающая выборка: rmse: 40.06133646080071 R2: 0.19809333896169845
Валидационная выборка: rmse: 40.038344412124424 R2: 0.20088239557946086


### Анализ полученнных результатов

**Отрицательные предсказания**

Изучим кол-во отрицательных предсказаний моделей, ведь отрицательное предсказание говорит о неточностях моделей, т.к. получение подобных результатов противоречит здравому смыслу. Затем заменим замену отрицательных предсказаний на 0.

In [12]:
print('Кол-во отрицательных предсказаний модели для 1ого региона: {}'.format(
    (predictions_0 < 0).sum())
     )
print('Кол-во отрицательных предсказаний модели для 2ого региона: {}'.format(
    (predictions_1 < 0).sum())
     )
print('Кол-во отрицательных предсказаний модели для 3его региона: {}'.format(
    (predictions_2 < 0).sum())
     )


Кол-во отрицательных предсказаний модели для 1ого региона: 2
Кол-во отрицательных предсказаний модели для 2ого региона: 515
Кол-во отрицательных предсказаний модели для 3его региона: 0


In [13]:
predictions_0 = predictions_0.apply(lambda x: x if x > 0 else 0)
predictions_1 = predictions_1.apply(lambda x: x if x > 0 else 0)
predictions_2 = predictions_2.apply(lambda x: x if x > 0 else 0)

**Средний объем сырья** 

In [14]:
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 1ом регионе'.format(predictions_0.mean()))
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 2ом регионе'.format(predictions_1.mean()))
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 3ем регионе'.format(predictions_2.mean()))

Средний объем сырья в месторождениях в результате предсказания модели: 92.45 тыс.бар. в 1ом регионе
Средний объем сырья в месторождениях в результате предсказания модели: 68.67 тыс.бар. в 2ом регионе
Средний объем сырья в месторождениях в результате предсказания модели: 94.93 тыс.бар. в 3ем регионе


Изучим взаимосвязь между предсказаниями и признаками моделей. Для этого построим матрицу корреляции

In [14]:
datas=[data_0,data_1,data_2]
k = 0 
for data in datas:
    k+=1
    print('регион {}'.format(str(k)))
    display(data.corr())

регион 1


,f0,f1,f2,product
f0,1.000000,-0.440700,-0.003122,0.143563
f1,-0.440700,1.000000,0.001691,-0.192379
f2,-0.003122,0.001691,1.000000,0.483645
product,0.143563,-0.192379,0.483645,1.000000


регион 2


,f0,f1,f2,product
f0,1.000000,0.182273,-0.001753,-0.030467
f1,0.182273,1.000000,-0.002589,-0.010148
f2,-0.001753,-0.002589,1.000000,0.999397
product,-0.030467,-0.010148,0.999397,1.000000


регион 3


,f0,f1,f2,product
f0,1.000000,0.000515,-0.000460,-0.001981
f1,0.000515,1.000000,0.000837,-0.000968
f2,-0.000460,0.000837,1.000000,0.445860
product,-0.001981,-0.000968,0.445860,1.000000


#### Вывод
Для второго региона наблюдается почти идеальная корреляция признака f2 с целевым признаком, что и объясняет такую точность в предсказаниях для 2ого региона.

### Вывод
Как видно из полученных результатов обучения и проверки моделей видно, что модель хорошо работает только для второго региона, для него R2 стремится к 1. RMSE примерно равно 0.89 тыс.бар. на тестовой и обучающей выборках, при среднем значении на обучающей выборке 68.8 тыс.бар.

Для первого и третьего региона таких хороших соответствий между предсказаниями модели и реальными значениям не наблюдается. Для них rmse на уровне 37-40 тыс.бар. при средних значениях 92-94 тыс.бар. Что свидетельсвует об очень больших неточностях в предсказаниях модели.

Также было замечено, что в предсказаниях моделей присутствуют и отрицательные значения объемов запасов нефти. Отсюда возникает вопрос как правильно интерпретировать такие результаты? Мне кажется в данном случае более корректно заменить отрицательные предсказания на 0 (хотя это и не окажет сильного влияния на конечный результат, т.к. таких месторождений мало по отношению к положительным прогнозам)

## Подготовка к расчёту прибыли

Условия задачи:
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые). При разведке региона проводится исследование 500 точек. Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей. Один баррель сырья приносит 4500 рублей прибыли. Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.

#### Объявление констант

In [15]:
NUMBER_OF_POINTS = 500 # кол-во точек для исследования
BUDGET = 10**10 # бюджет
EXPENSES_PER_POINT = 50*(10**6) # стоимость бурения одной скважины 
PROFIT_PER_BARREL = 4500 # стоимость одного барреля

In [16]:
Number_of_used_points = BUDGET / EXPENSES_PER_POINT
print('Кол-во месторождений, которые будут использоваться для добычи: {}'.format(int(Number_of_used_points)))

Кол-во месторождений, которые будут использоваться для добычи: 200


#### Посчитайте минимальный средний объём сырья в месторождениях региона, достаточный для его разработки. Напишите выводы.


In [17]:
Min_mean_product = BUDGET/(Number_of_used_points*PROFIT_PER_BARREL*1000)
print('Минимальный средний объём сырья в месторождениях региона: {:.2f} тыс.бар.'.format(Min_mean_product))

Минимальный средний объём сырья в месторождениях региона: 11.11 тыс.бар.


#### Предсказанные значения

In [18]:
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 1ом регионе'.format(predictions_0.mean()))
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 2ом регионе'.format(predictions_1.mean()))
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 3ем регионе'.format(predictions_2.mean()))

Средний объем сырья в месторождениях в результате предсказания модели: 92.45 тыс.бар. в 1ом регионе
Средний объем сырья в месторождениях в результате предсказания модели: 68.67 тыс.бар. в 2ом регионе
Средний объем сырья в месторождениях в результате предсказания модели: 94.93 тыс.бар. в 3ем регионе


#### Истинные значения

In [20]:
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 1ом регионе'.format(target_valid_0.mean()))
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 2ом регионе'.format(target_valid_1.mean()))
print('Средний объем сырья в месторождениях в результате предсказания модели: {:.2f} тыс.бар. в 3ем регионе'.format(target_valid_2.mean()))

Средний объем сырья в месторождениях в результате предсказания модели: 92.32 тыс.бар. в 1ом регионе
Средний объем сырья в месторождениях в результате предсказания модели: 68.66 тыс.бар. в 2ом регионе
Средний объем сырья в месторождениях в результате предсказания модели: 95.00 тыс.бар. в 3ем регионе


#### Вывод
Объем минимального среднего объема сырья в месторождениях при котором разработка будет неубыточной - 11.11 тыс.бар. Если сравнивать это значение с реальным средним объемом запаса сырья в месторождениях в регионах, то оно сильно меньше.  

#### Напишите функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.

In [19]:
def profit(target, predictions):
    count = int(Number_of_used_points)
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:count]
    sum_profit = (selected*PROFIT_PER_BARREL *1000).sum() - BUDGET
    return sum_profit

#### Комментарий
Функция profit считает прибыль, получаемую от освоения 200 лучших месторождений из тех, что подаются на вход (по условиям задачи разведывается 500 месторождений). Отбор 200 лучших происходит по результатам предсказаний модели. Из суммарной прибыли вычитается средства потраченные на бурение этих 200 месторождений.

## Расчёт прибыли и рисков 

In [20]:
def bootstrap_profit(target, predictions):
    predictions = pd.Series(predictions)
    target = pd.Series(target)
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n = int(NUMBER_OF_POINTS), 
                                               replace = True, random_state = state)
        predictions_subsample = predictions[target_subsample.index]
        values.append(profit(target, predictions_subsample))
    values = pd.Series(values)
    mean = values.mean()
    print('Среднее значение прибыли по выборкам: {:.2f} млрд'.format(mean/(10**9)))
    confidence_interval = st.t.interval(0.95, 
                                 df = len(values) - 1, 
                                 loc = values.mean(),
                                 scale = values.sem()
                                 )
    print("95%-ый доверительный интервал: {:.2f} млрд. - {:.2f} млрд.".format(confidence_interval[0]/(10**9),confidence_interval[1]/(10**9)))
    print("Риск : {}".format(st.percentileofscore(values,0)))
    return values

### Посчитайте риски и прибыль для каждого региона:
 - 4.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 - 4.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков.
 - 4.3. Напишите выводы: предложите регион для разработки месторождений и обоснуйте выбор.

#### Комментарий
Функция bootstrap_profit - применяет технику bootstrap с 1000 выборок для подсчета среднего значения прибыли, поиска 95% доверительного интервала и риска убытков. Риск убытков считается как вероятность получить отрицательное значение прибыли(В данной задаче для всех регионов такой риск оказался 0).

In [21]:
values_0 = bootstrap_profit(target_valid_0.reset_index(drop = True),predictions_0)

Среднее значение прибыли по выборкам: 93.93 млрд
95%-ый доверительный интервал: 93.77 млрд. - 94.09 млрд.
Риск : 0.0


In [22]:
values_1 = bootstrap_profit(target_valid_1.reset_index(drop = True),predictions_1)

Среднее значение прибыли по выборкам: 94.48 млрд
95%-ый доверительный интервал: 94.35 млрд. - 94.60 млрд.
Риск : 0.0


In [23]:
values_2 = bootstrap_profit(target_valid_2.reset_index(drop = True),predictions_2)

Среднее значение прибыли по выборкам: 93.62 млрд
95%-ый доверительный интервал: 93.45 млрд. - 93.78 млрд.
Риск : 0.0


### Вывод
Как видно из полученных результатов, все регионы являются прибыльными согласно проведенным расчетам. Но так как цель состояла в поиске наиболее прибыльного региона,то значит необходимо выбрать 2ой регион, т.к. он принесет наибольшую прибыль.

## Общий вывод
В ходе выполнения работы были достигнуты все поставленные задачи. В качестве региона для добычи был выбран 2ой регион, т.к. он потенциально будет самым прибыльным.

